# Two-Stage Object detection w/ Freeze Learning & Fine-tuning
For this approach, it will be used a pretrained DensetNet201 as a backbone for a F-RCNN model for object detection, to which it will be applied freeze learning and fine-tuning to our task.

In [3]:
import os

DATA_DIR = "../data/"
IMG_DIR = DATA_DIR + "/images/"
ANNOTATION_DIR = DATA_DIR + "/annotations/"
SPLITS_DIR = DATA_DIR + "/dl-split/"
OUT_DIR = "./out/yolo_obj_detect/"

os.makedirs(OUT_DIR, exist_ok=True)

SEED = 42

# Model Definition

In [4]:
!git clone https://github.com/ultralytics/yolov5 models/yolov5

Cloning into 'models/yolov5'...
remote: Enumerating objects: 12192, done.
remote: Total 12192 (delta 0), reused 0 (delta 0), pack-reused 12192
Receiving objects: 100% (12192/12192), 11.95 MiB | 670.00 KiB/s, done.
Resolving deltas: 100% (8441/8441), done.


In [5]:
import numpy as np
from torch.utils.data import SubsetRandomSampler

np.random.seed(SEED)

train_names = []
with open('../data/dl-split/train.txt', 'r') as train_f:
    train_names = train_f.readlines()
    train_names = [x.strip() for x in train_names]

train_indices = list(range(len(train_names)))
np.random.shuffle(train_indices)
train_val_split = int(np.floor(0.2 * len(train_indices)))

train_idx, val_idx = train_indices[train_val_split:], train_indices[:train_val_split]

val_data = [train_names[idx] for idx in val_idx]
train_data = [train_names[idx] for idx in train_idx]

test_data = []
with open('../data/dl-split/test.txt', 'r') as test_f:
    test_data = test_f.readlines()
    test_data = [x.strip() for x in train_names]



# Prepare Folders

In [6]:
from utils.utils import parse_annotation
import shutil

TRAIN_DIR='../data/images/train'
TEST_DIR='../data/images/test'
VAL_DIR='../data/images/val'

TRAIN_LABELS='../data/labels/train'
TEST_LABELS='../data/labels/test'
VAL_LABELS='../data/labels/val'

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

os.makedirs(TRAIN_LABELS, exist_ok=True)
os.makedirs(TEST_LABELS, exist_ok=True)
os.makedirs(VAL_LABELS, exist_ok=True)

label_encode_map = {
    "trafficlight": 0,
    "speedlimit": 1,
    "crosswalk": 2,
    "stop": 3,
}

def transform_labels(out_path: str, annot_dict):
    annot_file = open(out_path, 'w')
    
    img_width = annot_dict['width']
    img_height = annot_dict['height']
    for label, box in zip(annot_dict['labels'], annot_dict['boxes']):
        width = (box[2]-box[0])/img_width
        height = (box[3]-box[1])/img_height
        
        x_center = (box[0]+box[2])/2/img_width
        y_center = (box[3]+box[1])/2/img_height
        
        annot_file.write(f'{label} {x_center} {y_center} {width} {height}\n')
    annot_file.close()

# training data
for name in train_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{TRAIN_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=TRAIN_DIR)
  
# validation data      
for name in val_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{VAL_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=VAL_DIR)
 
# test data       
for name in test_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{TEST_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=TEST_DIR)

# Train Model

In [10]:
!python models/yolov5/train.py -h

usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
                [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
                [--rect] [--resume [RESUME]] [--nosave] [--noval]
                [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--bucket BUCKET] [--cache [CACHE]] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls]
                [--optimizer {SGD,Adam,AdamW}] [--sync-bn] [--workers WORKERS]
                [--project PROJECT] [--name NAME] [--exist-ok] [--quad]
                [--cos-lr] [--label-smoothing LABEL_SMOOTHING]
                [--patience PATIENCE] [--freeze FREEZE [FREEZE ...]]
                [--save-period SAVE_PERIOD] [--local_rank LOCAL_RANK]
                [--entity ENTITY] [--upload_dataset [UPLOAD_DATASET]]
                [--bbox_interval BBOX_INTERVAL]
                [--artifact_alias ARTIFACT_ALIAS]

options:
  -h, --help            show this help message a

In [13]:
!python models/yolov5/train.py --batch 4 --epochs 3 --data models/yolo_cfg.yaml --workers 2 --project out/yolo/ --hyp models/hyp.yaml

train: weights=models/yolov5/yolov5s.pt, cfg=, data=models/yolo_cfg.yaml, hyp=models/hyp.yaml, epochs=3, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=out/yolo/, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5
/usr/lib/python3.10/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch